<a href="https://colab.research.google.com/github/jphall663/GWU_ML/blob/main/notebook/lecture_9/Lecture_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# License 
***
Copyright (C) 2017--2022 J. Patrick Hall, jphall@gwu.edu

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

1. Python Imports and Config

In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# to upload local files
import io
from google.colab import files  

# display matplotlib graphics in notebook
%matplotlib inline

# for better reproducibility
SEED = 12345

2. Install Java for h2o

In [ ]:
# install Java for h2o backend
!apt-get install default-jre
!java -version

3. Install h2o python package

In [ ]:
# install h2o
!pip install h2o 

4. Start h2o

In [ ]:
# start and connect to h2o server
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.kmeans import H2OKMeansEstimator
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator

h2o.init()

5. Upload example data

In [ ]:
# special google collab command to upload a file from computer
uploaded = files.upload() # REQUIRES STUDENT INPUT

In [ ]:
# 6
uploaded.keys() # what is stored in that Python object?

7. Define character column types carefully

In [ ]:
# define input variable measurement levels 
# strings automatically parsed as enums (nominal)
# numbers automatically parsed as numeric
col_types = {'bad_loan': 'enum',
             'GRP_addr_state': 'enum',
             'GRP_home_ownership': 'enum',
             'GRP_verification_status': 'enum',
             'GRP_REP_home_ownership': 'enum',
             'GRP_purpose': 'enum'}

8. Read data into h2o

In [ ]:
raw = pd.read_csv(io.StringIO(uploaded['loan_clean.csv'].decode('utf-8'))) # name in quotes here must match name in 6 above
frame = h2o.H2OFrame(raw, column_types=col_types)

9. Display a summary of the data

In [ ]:
frame.describe()

10. Assign basic modeling roles -- `y` will not be used

In [ ]:
# assign target and inputs
y = 'bad_loan'
X = [name for name in frame.columns if name not in ['id', '_WARN_', 'GRP_home_ownership', y]]
print(y)
print(X)
print(len(X))

11. Train a _k_-means model

In [ ]:
# train k-means cluster model
# data is already standardized
# w/ 3 clusters
# print summary
clusters = H2OKMeansEstimator(standardize=False, k=, seed=) # REQUIRES STUDENT INPUT
clusters.train(x=X, training_frame=frame)
print(clusters)

12. Display a few example cluster labels

In [ ]:
# join cluster labels to original data for further analysis
labels = clusters.predict(frame)
labeled_frame =  frame.cbind(labels)
labeled_frame.head(rows=) # REQUIRES STUDENT INPUT (scroll right to see cluster assignments)

13. Divide columns into numeric ("real") and categorical ("enum")

In [ ]:
# determine column types
reals, enums = [], []
for key, val in labeled_frame.types.items():
    if key in X:
        if val == 'enum':
            enums.append(key)
        else: 
            reals.append(key)

print(enums)
print(reals)

14. Profile clusters using means

In [ ]:
# profile clusters by means
grouped = labeled_frame.group_by(by=['predict'])
means = grouped.mean(col=reals).get_frame()
print(means)

15. Profile clusters using modes

In [ ]:
# profile clusters by modes
grouped = labeled_frame.group_by(by=['predict'])
modes = grouped.mode(col=enums).get_frame()
print(modes)

16. Define a custom function for plotting the clusters

In [ ]:
# define a function for plotting clusters in 2-d
def plot(_2d_labeled_frame):

    _0 = plt.scatter(features_pandas[_2d_labeled_frame.label == 0].iloc[0:750, 0], 
                     features_pandas[_2d_labeled_frame.label == 0].iloc[0:750, 1],
                     color='m', marker='^', alpha=.15)

    _1 = plt.scatter(features_pandas[_2d_labeled_frame.label == 1].iloc[0:750, 0], 
                     features_pandas[_2d_labeled_frame.label == 1].iloc[0:750, 1],
                     color='c', alpha=.15)

    _2 = plt.scatter(features_pandas[_2d_labeled_frame.label == 2].iloc[0:750, 0], 
                     features_pandas[_2d_labeled_frame.label == 2].iloc[0:750, 1],
                     color='g', marker='s', alpha=.15)               
                     
    plt.legend([_0, _1, _2], 
               ['Cluster 0', 'Cluster 1', 'Cluster 2'],
               bbox_to_anchor=(1.05, 0.0), 
               loc=3, borderaxespad=0.)
    
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')

17. Project clusters into a lower-dimensional space with PCA to visualize

In [ ]:
# project training data onto 2-D using principal components
# join with clusters labels
# plot
pca = H2OPrincipalComponentAnalysisEstimator(k=) # REQUIRES STUDENT INPUT
pca.train(x=X, training_frame=labeled_frame)
features = pca.predict(labeled_frame)
features_pandas = features.as_data_frame()
features_pandas['label'] = labeled_frame[-1].as_data_frame()
print(features_pandas.head())
plot(features_pandas)

18. Shutdown h2o

In [ ]:
# shutdown h2o
h2o.cluster().shutdown(prompt=False)